In [3]:
# Set seeds for reproducibility
import random
random.seed(69)

import numpy as np
np.random.seed(69)

import tensorflow as tf
tf.random.set_seed(69)
print(tf.__version__)

2.10.0


In [9]:
# Allow memory growth on the GPU
import tensorflow as tf
# Check if GPU is available
gpu_available = tf.config.list_physical_devices('GPU')
print("Available GPUs:", gpu_available)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)  # Memory growth must be set at program startup

Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import os
import random
import numpy as np
import json
from PIL import Image
from tensorflow.keras.models import load_model
import tensorflow as tf

# Function to load and preprocess the image
@tf.function  # Optional: This ensures efficient re-use and reduces retracing.
def load_and_preprocess_image(image_path, target_size=(224,224)):
    img = Image.open(image_path).convert('RGB')  # Ensure all images are RGB
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0  # Scale pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to predict the class of an image
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    
    # Accessing the predicted class name
    try:
        predicted_class_name = class_indices[str(predicted_class_index)]
    except KeyError:
        print(f"KeyError: {predicted_class_index} not found in class indices.")
        print("Available class indices:", list(class_indices.keys()))
        return None, predictions[0]

    confidence_scores = predictions[0]
    return predicted_class_name, confidence_scores

# Load the model and class indices
model = load_model('plant_disease_prediction_model_3.keras')
with open('class_indices.json') as f:
    class_indices = json.load(f)

# Create test_images directory if it doesn't exist
base_dir = 'plantvillage_dataset'
test_images_dir = os.path.join(base_dir, 'test_images')
os.makedirs(test_images_dir, exist_ok=True)

# Define the path to the color folder
color_folder_path = os.path.join(base_dir, 'color')

# List all class folders in the color dataset
class_folders = [f for f in os.listdir(color_folder_path) if os.path.isdir(os.path.join(color_folder_path, f))]

# Randomly pick an image from a random class folder within the color directory
selected_class_folder = random.choice(class_folders)
selected_folder_path = os.path.join(color_folder_path, selected_class_folder)
image_files = [f for f in os.listdir(selected_folder_path) if f.endswith('.JPG')]

if image_files:
    selected_image = random.choice(image_files)
    selected_image_path = os.path.join(selected_folder_path, selected_image)

    # Generate a random name for the new image
    random_name = f"test_image_{random.randint(1000, 9999)}.JPG"
    new_image_path = os.path.join(test_images_dir, random_name)

    # Copy the selected image to the test_images directory with the new name
    Image.open(selected_image_path).save(new_image_path)

    # Predict the class of the new image
    predicted_class_name, confidence_scores = predict_image_class(model, new_image_path, class_indices)

    # Output the result if prediction was successful
    if predicted_class_name:
        print("Predicted Class Name:", predicted_class_name)
        
        # Checking the guessed output
        actual_class_name = selected_class_folder  # The actual class from which the image was picked
        if predicted_class_name == actual_class_name:
            print("The predicted class is correct!")
        else:
            print(f"The predicted class is incorrect. Actual class: {actual_class_name}")

        # Sort confidence scores from highest to lowest
        sorted_confidence_scores = sorted(zip(class_indices.values(), confidence_scores), key=lambda x: x[1], reverse=True)

        # Display sorted confidence scores
        print("Confidence Scores (from highest to lowest):")
        for class_name, score in sorted_confidence_scores:
            print(f"{class_name}: {score:.6f}")  # Adjust the precision as needed
else:
    print("No images found in the selected class folder.")


OSError: No file or directory found at plant_disease_prediction_model_2.keras

In [12]:
from numba import cuda

# Select your GPU
cuda.select_device(0)  # Change the index if you have multiple GPUs
# Close the GPU context to free up memory
cuda.close()
